Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/forecasting-orange-juice-sales/auto-ml-forecasting-orange-juice-sales.png)

# Data Preparation
In this notebook, we synthetically generate a total of 12,222 orange juice sales datasets and leverage ParallelRunStep to build many models.

## Setup

In [1]:
import azureml.core
import pandas as pd
import numpy as np
import logging

from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig

/Users/jason.shi/anaconda3/envs/aml/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


As part of the setup you have already created a <b>Workspace</b>. To run AutoML, you also need to create an <b>Experiment</b>. An Experiment corresponds to a prediction problem you are trying to solve, while a Run corresponds to a specific approach to the problem. 

In [5]:
ws = Workspace(subscription_id="bbd86e7d-3602-4e6d-baa4-40ae2ad9303c", resource_group="ManyModelsSA", workspace_name="ManyModelsSAv1")
ws.get_details()
experiment_name = 'automl-ojforecasting'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['SKU'] = ws.sku
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
SDK version,1.0.74
Subscription ID,bbd86e7d-3602-4e6d-baa4-40ae2ad9303c
Workspace,ManyModelsSAv1
SKU,Enterprise
Resource Group,ManyModelsSA
Location,centralus
Run History Name,automl-ojforecasting


## Compute
You will need to create a [compute target](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute) for your AutoML run. In this tutorial, you create AmlCompute as your training compute resource.
#### Creation of AmlCompute takes approximately 5 minutes. 
If the AmlCompute with that name is already in your workspace this code will skip the creation process.
As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read this article on the default limits and how to request more quota.

In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# Choose a name for your cluster.
amlcompute_cluster_name = "cpu-cluster"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute_target = cts[amlcompute_cluster_name]
    
if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", # for GPU, use "STANDARD_NC6"
                                                                #vm_priority = 'lowpriority', # optional
                                                                max_nodes = 6)

    # Create the cluster.
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('Checking cluster status...')
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)
    
# For a more detailed view of current AmlCompute status, use get_status().

Found existing compute target.
Checking cluster status...
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


## Upload the raw data to the blob

In [18]:
dataDir='raw_data/'
ojsalesraw_path=dataDir+'dominicks_OJ.csv'

In [19]:
datastore = ws.get_default_datastore() 
datastore.upload_files([ojsalesraw_path], target_path='ojsalesraw', overwrite=False, show_progress=True)

Uploading an estimated of 1 files
Target already exists. Skipping upload for ojsalesraw/dominicks_OJ.csv
Uploaded 0 files


$AZUREML_DATAREFERENCE_f5d895badb8848bda3be455b450efef0

## Dowloand the raw data from blob

In [22]:
download_target_path = 'raw_data/'
datastore.download(target_path=download_target_path,
                   prefix='ojsalesraw/dominicks_OJ.csv',
                   show_progress=True)

WARNING - Path already exists. Skipping download for raw_data/ojsalesraw/dominicks_OJ.csv


0

## (optional) Register dataset 

In [24]:
from azureml.core.dataset import Dataset
datastore_paths = [
                  (datastore, 'ojsalesraw')
                 ]
ojsalesraw_ds = Dataset.File.from_files(path=datastore_paths)

In [27]:
ojsalesraw_ds = ojsalesraw_ds.register(workspace=ws, 
                                       name='ojsalesraw', 
                                       description='ojsales raw data')

## Synthetic data generation

In [4]:
time_column_name = 'WeekStarting'
data = pd.read_csv("dominicks_OJ.csv", parse_dates=[time_column_name])
data.head()

,WeekStarting,Store,Brand,Quantity,logQuantity,Advert,Price,Age60,COLLEGE,INCOME,Hincome150,Large HH,Minorities,WorkingWoman,SSTRDIST,SSTRVOL,CPDIST5,CPWVOL5
0,1990-06-14,2,dominicks,10560,9.26,1,1.59,0.23,0.25,10.55,0.46,0.10,0.11,0.30,2.11,1.14,1.93,0.38
1,1990-06-14,2,minute.maid,4480,8.41,0,3.17,0.23,0.25,10.55,0.46,0.10,0.11,0.30,2.11,1.14,1.93,0.38
2,1990-06-14,2,tropicana,8256,9.02,0,3.87,0.23,0.25,10.55,0.46,0.10,0.11,0.30,2.11,1.14,1.93,0.38
3,1990-06-14,5,dominicks,1792,7.49,1,1.59,0.12,0.32,10.92,0.54,0.10,0.05,0.41,3.80,0.68,1.60,0.74
4,1990-06-14,5,minute.maid,4224,8.35,0,2.99,0.12,0.32,10.92,0.54,0.10,0.05,0.41,3.80,0.68,1.60,0.74


In [233]:
data.describe()

,Store,Quantity,logQuantity,Advert,Price,Age60,COLLEGE,INCOME,Hincome150,Large HH,Minorities,WorkingWoman,SSTRDIST,SSTRVOL,CPDIST5,CPWVOL5
count,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00
mean,80.88,17312.21,9.17,0.24,2.28,0.17,0.23,10.62,0.34,0.12,0.16,0.36,5.10,1.21,2.12,0.44
std,35.58,27477.66,1.02,0.43,0.65,0.06,0.11,0.28,0.24,0.03,0.19,0.05,3.47,0.53,0.73,0.22
min,2.00,64.00,4.16,0.00,0.52,0.06,0.05,9.87,0.00,0.01,0.02,0.24,0.13,0.40,0.77,0.09
25%,53.00,4864.00,8.49,0.00,1.79,0.12,0.15,10.46,0.12,0.10,0.04,0.31,2.77,0.73,1.63,0.27
50%,86.00,8384.00,9.03,0.00,2.17,0.17,0.23,10.64,0.35,0.11,0.07,0.36,4.65,1.12,1.96,0.38
75%,111.00,17408.00,9.76,0.00,2.73,0.21,0.28,10.80,0.53,0.14,0.19,0.40,6.65,1.54,2.53,0.56
max,137.00,716416.00,13.48,1.00,3.87,0.31,0.53,11.24,0.92,0.22,1.00,0.47,17.86,2.57,4.11,1.14


In [139]:
import pandas as pd 
  
dates = pd.date_range(start ='1990-06-13',  
         end ='1992-10-02', freq ='W-THU') 

In [140]:
dates_df = pd.DataFrame(data=dates)

In [172]:
dates_df_all = pd.DataFrame({'WeekStarting':np.repeat(dates_df.values,11973)})

,WeekStarting
0,1990-06-14
1,1990-06-14
2,1990-06-14
3,1990-06-14
4,1990-06-14
5,1990-06-14
6,1990-06-14
7,1990-06-14
8,1990-06-14
9,1990-06-14


In [133]:
brand_data={'Brand':['dominicks', 'minute.maid', 'tropicana']}
brands_list = pd.DataFrame(data=brand_data)

,Brand
0,dominicks
1,minute.maid
2,tropicana


In [173]:
brands_df_all = pd.DataFrame()
brands_df_all = pd.concat([brands_list]*482911, ignore_index=True)

,Brand
0,dominicks
1,minute.maid
2,tropicana
3,dominicks
4,minute.maid
5,tropicana
6,dominicks
7,minute.maid
8,tropicana
9,dominicks


In [108]:
data.Store.unique()

array([  2,   5,   8,   9,  14,  18,  21,  28,  32,  33,  40,  44,  45,
        47,  48,  49,  50,  51,  52,  53,  54,  56,  59,  62,  64,  67,
        68,  70,  71,  72,  73,  75,  76,  77,  78,  80,  81,  83,  84,
        86,  88,  89,  90,  91,  92,  93,  94,  97, 101, 103, 104, 105,
       106, 107, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
       121, 122, 123, 124, 126, 128, 129, 132,  12,  74,  95,  98, 130,
       100, 102, 131, 137, 134])

In [196]:
store_list = np.arange(138,4129,1)

In [197]:
store_df = pd.DataFrame(data=store_list)

In [198]:
store_df_all = pd.DataFrame({'Store':np.repeat(store_df.values,3)})

,Store
0,138
1,138
2,138
3,139
4,139
5,139
6,140
7,140
8,140
9,141


In [199]:
store_df_final = pd.concat([store_df_all]*121, ignore_index=True)

,Store
0,138
1,138
2,138
3,139
4,139
5,139
6,140
7,140
8,140
9,141


In [183]:
advert=[1,0,0]
advert_df = pd.DataFrame({'Advert':advert})

,Advert
0,1
1,0
2,0


In [184]:
advert_df_all = pd.DataFrame()
advert_df_all = pd.concat([advert_df]*482911, ignore_index=True)

,Advert
0,1
1,0
2,0
3,1
4,0
5,0
6,1
7,0
8,0
9,1


In [219]:
df_combined = pd.concat([dates_df_all, store_df_final, brands_df_all], axis=1, sort=False)

In [354]:
df_combined['Quantity'] = np.random.randint(10000, 20000, df_combined.shape[0])
df_combined_final = pd.concat([df_combined,advert_df_all], axis=1)

In [355]:
import random
df_combined_final['Price'] = np.random.randint(190, 270, df_combined.shape[0])/100
df_combined_final

,WeekStarting,Store,Brand,Quantity,Advert,Price
0,1990-06-14,138,dominicks,8757,1,2.50
1,1990-06-14,138,minute.maid,14633,0,2.24
2,1990-06-14,138,tropicana,8558,0,2.61
3,1990-06-14,139,dominicks,6430,1,2.10
4,1990-06-14,139,minute.maid,5061,0,2.14
5,1990-06-14,139,tropicana,13655,0,2.51
6,1990-06-14,140,dominicks,5517,1,2.41
7,1990-06-14,140,minute.maid,2355,0,1.93
8,1990-06-14,140,tropicana,4224,0,1.97
9,1990-06-14,141,dominicks,15142,1,2.12


In [356]:
data_filtered = data[['WeekStarting', 'Store', 'Brand', 'Quantity','Advert', 'Price']]
frames = [data_filtered, df_combined_final]
df_final = pd.concat(frames)

In [357]:
data_filtered.describe()

,Store,Quantity,Advert,Price
count,28947.00,28947.00,28947.00,28947.00
mean,80.88,17312.21,0.24,2.28
std,35.58,27477.66,0.43,0.65
min,2.00,64.00,0.00,0.52
25%,53.00,4864.00,0.00,1.79
50%,86.00,8384.00,0.00,2.17
75%,111.00,17408.00,0.00,2.73
max,137.00,716416.00,1.00,3.87


In [358]:
df_final.describe()

,Store,Quantity,Advert,Price
count,1477680.00,1477680.00,1477680.00,1477680.00
mean,2092.80,8709.44,0.33,2.29
std,1175.69,6298.62,0.47,0.25
min,2.00,64.00,0.00,0.52
25%,1075.00,4320.00,0.00,2.09
50%,2093.00,8537.00,0.00,2.29
75%,3111.00,12794.00,1.00,2.50
max,4128.00,716416.00,1.00,3.87


In [359]:
df_final['Revenue'] = df_final['Quantity']*df_final['Price'] 

In [360]:
df_final.tail()

,WeekStarting,Store,Brand,Quantity,Advert,Price,Revenue
1448728,1992-10-01,4127,minute.maid,8059,0,2.09,16843.31
1448729,1992-10-01,4127,tropicana,6816,0,1.97,13427.52
1448730,1992-10-01,4128,dominicks,5831,1,2.28,13294.68
1448731,1992-10-01,4128,minute.maid,11555,0,2.00,23110.00
1448732,1992-10-01,4128,tropicana,9579,0,2.45,23468.55


In [361]:
dfs = [x for _, x in df_final.groupby(['Store', 'Brand'])]

In [362]:
df_final.count()

WeekStarting    1477680
Store           1477680
Brand           1477680
Quantity        1477680
Advert          1477680
Price           1477680
Revenue         1477680
dtype: int64

In [363]:
synthetic_path = 'synthetic_data/'
syn_filename='ojsales.csv'

df_final.to_csv(path_or_buf=synthetic_path+syn_filename,index=False)

In [364]:
ojsalessynthetic_path='synthetic_data'
datastore.upload(src_dir=ojsalessynthetic_path,
                 target_path='ojsalessynthetic',
                 overwrite=True,
                 show_progress=True)

Uploading an estimated of 2 files
Uploading synthetic_data/.DS_Store
Uploading synthetic_data/ojsales.csv
Uploaded synthetic_data/.DS_Store, 1 files out of an estimated total of 2
Uploaded synthetic_data/ojsales.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_028e1440013d4a1d8cec79a6c876eef6

In [365]:
refined_path = 'refined_data/'

for i in dfs:
    file_name = 'Store'+str(i['Store'].unique()).lstrip("['").rstrip("']")+'_'+str(i['Brand'].unique()).lstrip("['").rstrip("']")+'.csv'
    i.to_csv(path_or_buf=refined_path+file_name, index=False,)

In [366]:
ojsalesrefined_path='refined_data'
datastore.upload(src_dir=ojsalesrefined_path,
                 target_path='ojsalesrefined',
                 overwrite=True,
                 show_progress=True)

Uploading an estimated of 12223 files
Uploading refined_data/Store1000_dominicks.csv
Uploading refined_data/Store1004_dominicks.csv
Uploading refined_data/Store1013_dominicks.csv
Uploading refined_data/Store1014_minute.maid.csv
Uploading refined_data/Store1029_minute.maid.csv
Uploading refined_data/Store104_dominicks.csv
Uploading refined_data/Store1053_minute.maid.csv
Uploading refined_data/Store1064_tropicana.csv
Uploading refined_data/Store1073_tropicana.csv
Uploading refined_data/Store1080_dominicks.csv
Uploading refined_data/Store1097_dominicks.csv
Uploading refined_data/Store1121_minute.maid.csv
Uploading refined_data/Store1123_tropicana.csv
Uploading refined_data/Store1134_tropicana.csv
Uploading refined_data/Store113_dominicks.csv
Uploading refined_data/Store1143_dominicks.csv
Uploading refined_data/Store1150_dominicks.csv
Uploading refined_data/Store1154_dominicks.csv
Uploading refined_data/Store1166_minute.maid.csv
Uploading refined_data/Store116_minute.maid.csv
Uploading ref

Uploaded refined_data/Store1026_minute.maid.csv, 114 files out of an estimated total of 12223
Uploaded refined_data/Store1026_dominicks.csv, 115 files out of an estimated total of 12223
Uploading refined_data/Store1036_minute.maid.csv
Uploading refined_data/Store1036_tropicana.csv
Uploading refined_data/Store1037_dominicks.csv
Uploaded refined_data/Store1028_tropicana.csv, 116 files out of an estimated total of 12223
Uploaded refined_data/Store1025_dominicks.csv, 117 files out of an estimated total of 12223
Uploaded refined_data/Store1028_minute.maid.csv, 118 files out of an estimated total of 12223
Uploading refined_data/Store1037_minute.maid.csv
Uploading refined_data/Store1037_tropicana.csv
Uploaded refined_data/Store1028_dominicks.csv, 119 files out of an estimated total of 12223
Uploading refined_data/Store1038_dominicks.csv
Uploaded refined_data/Store102_tropicana.csv, 120 files out of an estimated total of 12223
Uploaded refined_data/Store1029_tropicana.csv, 121 files out of an 

Uploaded refined_data/Store1049_minute.maid.csv, 185 files out of an estimated total of 12223
Uploaded refined_data/Store104_tropicana.csv, 186 files out of an estimated total of 12223
Uploading refined_data/Store1058_tropicana.csv
Uploading refined_data/Store1059_dominicks.csv
Uploading refined_data/Store1059_minute.maid.csv
Uploaded refined_data/Store1051_minute.maid.csv, 187 files out of an estimated total of 12223
Uploaded refined_data/Store1052_dominicks.csv, 188 files out of an estimated total of 12223
Uploaded refined_data/Store104_minute.maid.csv, 189 files out of an estimated total of 12223
Uploaded refined_data/Store1049_tropicana.csv, 190 files out of an estimated total of 12223
Uploaded refined_data/Store1050_dominicks.csv, 191 files out of an estimated total of 12223
Uploaded refined_data/Store1048_tropicana.csv, 192 files out of an estimated total of 12223
Uploaded refined_data/Store1050_minute.maid.csv, 193 files out of an estimated total of 12223
Uploaded refined_data/S

Uploaded refined_data/Store1073_dominicks.csv, 263 files out of an estimated total of 12223
Uploading refined_data/Store1083_dominicks.csv
Uploaded refined_data/Store1073_minute.maid.csv, 264 files out of an estimated total of 12223
Uploaded refined_data/Store1072_tropicana.csv, 265 files out of an estimated total of 12223
Uploading refined_data/Store1083_minute.maid.csv
Uploading refined_data/Store1083_tropicana.csv
Uploaded refined_data/Store1074_minute.maid.csv, 266 files out of an estimated total of 12223
Uploading refined_data/Store1084_dominicks.csv
Uploaded refined_data/Store1074_tropicana.csv, 267 files out of an estimated total of 12223
Uploaded refined_data/Store1076_minute.maid.csv, 268 files out of an estimated total of 12223
Uploading refined_data/Store1084_minute.maid.csv
Uploading refined_data/Store1084_tropicana.csv
Uploaded refined_data/Store1075_dominicks.csv, 269 files out of an estimated total of 12223
Uploaded refined_data/Store1074_dominicks.csv, 270 files out of 

Uploaded refined_data/Store1098_dominicks.csv, 333 files out of an estimated total of 12223
Uploaded refined_data/Store1097_tropicana.csv, 334 files out of an estimated total of 12223
Uploaded refined_data/Store1099_minute.maid.csv, 335 files out of an estimated total of 12223
Uploaded refined_data/Store1095_tropicana.csv, 336 files out of an estimated total of 12223
Uploading refined_data/Store1105_tropicana.csv
Uploaded refined_data/Store1097_minute.maid.csv, 337 files out of an estimated total of 12223
Uploading refined_data/Store1106_dominicks.csv
Uploading refined_data/Store1106_minute.maid.csv
Uploading refined_data/Store1106_tropicana.csv
Uploaded refined_data/Store1096_dominicks.csv, 338 files out of an estimated total of 12223
Uploaded refined_data/Store1094_tropicana.csv, 339 files out of an estimated total of 12223
Uploading refined_data/Store1107_dominicks.csv
Uploaded refined_data/Store109_dominicks.csv, 340 files out of an estimated total of 12223
Uploading refined_data/S

...

Uploaded refined_data/Store916_tropicana.csv, 11947 files out of an estimated total of 12223
Uploaded refined_data/Store916_dominicks.csv, 11948 files out of an estimated total of 12223
Uploaded refined_data/Store914_tropicana.csv, 11949 files out of an estimated total of 12223
Uploading refined_data/Store924_tropicana.csv
Uploading refined_data/Store925_dominicks.csv
Uploading refined_data/Store925_minute.maid.csv
Uploaded refined_data/Store915_minute.maid.csv, 11950 files out of an estimated total of 12223
Uploading refined_data/Store925_tropicana.csv
Uploading refined_data/Store926_dominicks.csv
Uploading refined_data/Store926_minute.maid.csv
Uploaded refined_data/Store918_dominicks.csv, 11951 files out of an estimated total of 12223
Uploading refined_data/Store926_tropicana.csv
Uploaded refined_data/Store918_tropicana.csv, 11952 files out of an estimated total of 12223
Uploading refined_data/Store927_dominicks.csv
Uploaded refined_data/Store919_minute.maid.csv, 11953 files out of a

Uploaded refined_data/Store938_minute.maid.csv, 12017 files out of an estimated total of 12223
Uploaded refined_data/Store938_dominicks.csv, 12018 files out of an estimated total of 12223
Uploading refined_data/Store946_tropicana.csv
Uploaded refined_data/Store937_tropicana.csv, 12019 files out of an estimated total of 12223
Uploading refined_data/Store947_dominicks.csv
Uploaded refined_data/Store939_minute.maid.csv, 12020 files out of an estimated total of 12223
Uploaded refined_data/Store937_dominicks.csv, 12021 files out of an estimated total of 12223
Uploaded refined_data/Store939_dominicks.csv, 12022 files out of an estimated total of 12223
Uploaded refined_data/Store934_tropicana.csv, 12023 files out of an estimated total of 12223
Uploaded refined_data/Store938_tropicana.csv, 12024 files out of an estimated total of 12223
Uploading refined_data/Store947_minute.maid.csv
Uploading refined_data/Store947_tropicana.csv
Uploading refined_data/Store948_dominicks.csv
Uploading refined_da

Uploaded refined_data/Store961_dominicks.csv, 12098 files out of an estimated total of 12223
Uploaded refined_data/Store961_minute.maid.csv, 12099 files out of an estimated total of 12223
Uploading refined_data/Store971_minute.maid.csv
Uploaded refined_data/Store961_tropicana.csv, 12100 files out of an estimated total of 12223
Uploading refined_data/Store971_tropicana.csv
Uploading refined_data/Store972_dominicks.csv
Uploading refined_data/Store972_minute.maid.csv
Uploaded refined_data/Store962_minute.maid.csv, 12101 files out of an estimated total of 12223
Uploaded refined_data/Store964_minute.maid.csv, 12102 files out of an estimated total of 12223
Uploaded refined_data/Store962_dominicks.csv, 12103 files out of an estimated total of 12223
Uploading refined_data/Store972_tropicana.csv
Uploading refined_data/Store973_dominicks.csv
Uploading refined_data/Store973_minute.maid.csv
Uploaded refined_data/Store963_tropicana.csv, 12104 files out of an estimated total of 12223
Uploaded refine

Uploaded refined_data/Store986_minute.maid.csv, 12173 files out of an estimated total of 12223
Uploading refined_data/Store994_tropicana.csv
Uploaded refined_data/Store986_dominicks.csv, 12174 files out of an estimated total of 12223
Uploading refined_data/Store995_dominicks.csv
Uploaded refined_data/Store985_dominicks.csv, 12175 files out of an estimated total of 12223
Uploading refined_data/Store995_minute.maid.csv
Uploaded refined_data/Store986_tropicana.csv, 12176 files out of an estimated total of 12223
Uploaded refined_data/Store985_minute.maid.csv, 12177 files out of an estimated total of 12223
Uploaded refined_data/Store985_tropicana.csv, 12178 files out of an estimated total of 12223
Uploaded refined_data/Store987_dominicks.csv, 12179 files out of an estimated total of 12223
Uploading refined_data/Store995_tropicana.csv
Uploading refined_data/Store996_dominicks.csv
Uploaded refined_data/Store988_tropicana.csv, 12180 files out of an estimated total of 12223
Uploaded refined_dat

$AZUREML_DATAREFERENCE_8bcdab7f0d454a8ebcf8e55a30684925